In [44]:
import numpy as np
import tensorflow as tf
from pandas import DataFrame
from tensorflow import keras
from tensorflow.keras import layers

In [47]:
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data'
column_names = ['MPG', 'Cylinders', 'Displacement', 'Horsepower', 'Weight',
                'Acceleration', 'Model Year', 'Origin']

data = pd.read_csv(url, names=column_names,
                          na_values='?', comment='\t',
                          sep=' ', skipinitialspace=True)

In [46]:
def train_test_features(df: DataFrame) -> tuple:
    train_dataset = df.sample(frac=0.8, random_state=0)
    test_dataset = df.drop(train_dataset.index)

    train_features = train_dataset.copy()
    test_features = test_dataset.copy()

    return train_features, test_features


def train_test_labels(data) -> tuple:
    train_features, test_features = train_test_features(data)
    train_labels = train_features.pop('MPG')
    test_labels = test_features.pop('MPG')

    return train_labels, test_labels


def normalize_features(train_features):
    normalizer = tf.keras.layers.Normalization(axis=-1)
    normalizer.adapt(np.array(train_features))
    return normalizer

In [48]:
train_features, test_features = train_test_features(data)
train_labels, test_labels = train_test_labels(data)

In [56]:
normalizer = normalize_features(train_features)

In [58]:
print(normalizer.mean.numpy())

[[  23.591    5.428  193.061      nan 2963.824   15.596   75.947    1.553]]


In [59]:
first = np.array(train_features[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[  14.    8.  351.  153. 4129.   13.   72.    1.]]

Normalized: [[-1.21  1.53  1.52   nan  1.38 -0.93 -1.07 -0.69]]


In [55]:
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(64, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(1)
    ])

    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(0.001))
    return model

In [57]:
build_and_compile_model(normalizer)